In [1]:
import os
import numpy as np
import xgboost as xgb
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

In [3]:
#Define the directories and make sure the output exists
landsat_directory = "/Volumes/Seagate Portable Drive/Central_Park_Climate/Streamlined_Outputs/Step_Four_Processed_Landsat"
hls_directory = "/Volumes/Seagate Portable Drive/Central_Park_Climate/Streamlined_Outputs/Step_Four_Processed_HLS"
output_directory = "/Volumes/Seagate Portable Drive/Central_Park_Climate/Streamlined_Outputs/Temp"

os.makedirs(output_directory, exist_ok = True)

The Landsat ARD files are formatted like so: 

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': None, 'width': 75, 'height': 94, 'count': 6, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32618"]]'), 'transform': Affine(480.0, 0.0, 571980.0,
       0.0, -480.0, 4536208.046400001)}


and the HLS files are formatted like so:
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 1200, 'height': 1504, 'count': 7, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32618"]]'), 'transform': Affine(30.0, 0.0, 571980.0,
       0.0, -30.0, 4536208.046400001)}

In [ ]:
#Define a function to read the raster data
def read_raster(file_path):
    with rasterio.open(file_path) as src:
        return src.read().reshape(src.count, -1).T